### Comparison of the position of an ephemeris object with astropy 

Note: jplephem module - default is DE430 while current JPL-Horizons query is based on DE440s.
casacore measures: DE200 and DE405

(from ssd.pl.nasa.gov/planets/eph_export.html)
### DE200
Created September 1981; includes nutations but no librations.
Referred to the dynamical equator and equinox of 2000.
Covers JED 2305424.5 (1599 DEC 09)  to  JED 2513360.5 (2169 MAR 31).


### DE405
Created May 1997; includes both nutations and librations.
Referred to the International Celestial Reference Frame.
Covers JED 2305424.50  (1599 DEC 09)  to  JED 2525008.50  (2201 FEB 20)

### DE430
Created April 2013; includes librations and 1980 nutation.
Referred to the International Celestial Reference Frame version 2.0.
Covers JED 2287184.5, (1549 DEC 21) to JED 2688976.5, (2650 JAN 25).

### DE440
Created June 2020; compared to DE430, about 7 years of new data have
beend added.
Referred to the International Celestial Reference Frame version 3.0.
Covers JED 2287184.5, (1549 DEC 31) to JED 2688976.5, (2650 JAN 25).

In [1]:
from astropy.time import Time
from astropy.coordinates import solar_system_ephemeris, EarthLocation
from astropy.coordinates import get_body_barycentric, get_body, SkyCoord
import astropy.units as u
from casatools import table, quanta
tb = table()
qa = quanta()

In [3]:
# import helper functions defined in a separate notebook
from ipynb.fs.defs.EphemerisObjectImagingDemoFunctions import *

In [4]:
# venus_ephem_test.ms mosaic
# obs_date - 2019/01/08/11:21:50.208000
from casatasks import imhead
imgname = 'mosaic-mfs-exteph-venus'
iminfo = imhead(imgname+'.image',mode='list')
print(iminfo['date-obs'])

2019/01/08/11:21:50.208000


In [5]:
#reformat time for astropy
t = Time('2019-01-08 11:21:50.208000')

In [6]:
loc = EarthLocation.of_site('ALMA')
print(loc)

(2225015.30883296, -5440016.41799762, -2481631.27428014) m


In [7]:
with solar_system_ephemeris.set('builtin'):
    astropy_venus = get_body('venus', t, loc)

In [8]:
print(astropy_venus)

<SkyCoord (GCRS: obstime=2019-01-08 11:21:50.208, obsgeoloc=(-5084912.71740542, -2955268.70972292, -2472451.22172534) m, obsgeovel=(215.50598977, -370.46896037, -0.40195799) m / s): (ra, dec, distance) in (deg, deg, AU)
    (239.3651525, -16.96338202, 0.69134182)>


In [9]:
extephemtabpath='Venus_ICRS_JPL-Horizons_MJD58491.4.tab'
intephemtabpath = './venus_ephem_test.ms/FIELD/EPHEM0_Venus_58491.4.tab'

In [10]:
# find the direction of Venus at reftm
exteph_dir = ephem_dir(extephemtabpath, iminfo['date-obs'], 'ALMA')
inteph_dir = ephem_dir(intephemtabpath, iminfo['date-obs'], 'ALMA')

In [11]:
print('internal ephem:',qa.time(inteph_dir['m0'],prec=9)[0],qa.angle(inteph_dir['m1'],prec=9)[0])
print('external ephem:',qa.time(exteph_dir['m0'],prec=9)[0],qa.angle(exteph_dir['m1'],prec=9)[0])
print('astropy(builtin):',qa.time(qa.quantity(astropy_venus.ra.deg, 'deg'), prec=9)[0], qa.angle(qa.quantity(astropy_venus.dec.deg, 'deg'), prec=9)[0])

internal ephem: 15:57:28.746 -016.57.50.059
external ephem: 15:57:28.352 -016.57.51.782
astropy(builtin): 15:57:27.637 -016.57.48.175


In [12]:
!pip install jplephem

Looking in indexes: https://casa-pip.nrao.edu/repository/pypi-group/simple, https://pypi.org/simple
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.2/47.2 kB 627.6 kB/s eta 0:00:001m2.3 MB/s eta 0:00:01


In [13]:
solar_system_ephemeris.set('jpl')  

<ScienceState solar_system_ephemeris: 'jpl'>

In [14]:
de440_venus = get_body('venus', t, loc, ephemeris='de440')

In [15]:
print(de440_venus)

<SkyCoord (GCRS: obstime=2019-01-08 11:21:50.208, obsgeoloc=(-5084912.71740542, -2955268.70972292, -2472451.22172534) m, obsgeovel=(215.50598977, -370.46896037, -0.40195799) m / s): (ra, dec, distance) in (deg, deg, km)
    (239.36485145, -16.96313852, 1.03422641e+08)>


In [16]:
print('astropy(de440):',qa.time(qa.quantity(de440_venus.ra.deg, 'deg'), prec=9)[0], qa.angle(qa.quantity(de440_venus.dec.deg, 'deg'), prec=9)[0])

astropy(de440): 15:57:27.564 -016.57.47.299


In [17]:
#from casatools import measures
_me = measures()
_me.doframe(_me.observatory('ALMA'))
_me.doframe(_me.epoch('utc',iminfo['date-obs']))
meas_dir = _me.measure(_me.measure(_me.direction('VENUS'), 'AZELGEO'),'ICRS')
#print('measure:',qa.time(meas_dir['m0'],prec=9)[0],qa.angle(meas_dir['m1'],prec=9)[0])


In [18]:
print('internal ephem:',qa.time(inteph_dir['m0'],prec=9)[0],qa.angle(inteph_dir['m1'],prec=9)[0])
print('external ephem:',qa.time(exteph_dir['m0'],prec=9)[0],qa.angle(exteph_dir['m1'],prec=9)[0])
print('astropy(builtin):',qa.time(qa.quantity(astropy_venus.ra.deg, 'deg'), prec=9)[0], qa.angle(qa.quantity(astropy_venus.dec.deg, 'deg'), prec=9)[0])
print('astropy(de440):',qa.time(qa.quantity(de440_venus.ra.deg, 'deg'), prec=9)[0], qa.angle(qa.quantity(de440_venus.dec.deg, 'deg'), prec=9)[0])
print('measure:',qa.time(meas_dir['m0'],prec=9)[0],qa.angle(meas_dir['m1'],prec=9)[0])

internal ephem: 15:57:28.746 -016.57.50.059
external ephem: 15:57:28.352 -016.57.51.782
astropy(builtin): 15:57:27.637 -016.57.48.175
astropy(de440): 15:57:27.564 -016.57.47.299
measure: 15:57:28.543 -016.57.49.411


In [24]:
_me.separation(inteph_dir,exteph_dir)

{'unit': 'deg', 'value': 0.0016423736421583208}